### Import packages and log in to plotly

In [1]:
import numpy as np
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff

import terminal_optimization.visualisation        as visualisation
import terminal_optimization.forecast             as forecast
import terminal_optimization.financial_analysis   as financial

In [2]:
# Log in to Plotly servers
import plotly
plotly.tools.set_credentials_file(username='wijzermans', api_key='FKGDvSah3z5WCNREBZEq')
#plotly.tools.set_credentials_file(username='wijnandijzermans', api_key='xeDEwwpCK3aLLR4TIrM9')

# (*) To communicate with Plotly's server, sign in with credentials file
import plotly.plotly as py  

# (*) Useful Python/Plotly tools
import plotly.tools as tls   

# (*) Graph objects to piece together plots
from plotly.graph_objs import *

import matplotlib.pyplot as plt # (*) import matplotlib

%matplotlib inline

### Traffic projections

In [3]:
###################################################################################################
# Traffic projections on which estimate is based
###################################################################################################

# Import commodities from package
maize = forecast.bulk_commodities(**forecast.maize_data)

# Wheat - Probabilistic demand
historic_demand_maize = [1000000,1200000,1180000,1210000,1250000]
rate_maize  = 1.02
mu_maize    = 0.01          # avg bonus rate added to base rate (% points)  - input for random method
sigma_maize = 0.025         # standard deviation of bonus rate (% points)   - input for random method

# Create traffic projections
start_year = 2018
simulation_window = 20
traffic_projections = []
for iterations in range(1001):
    maize.random_scenario(start_year, simulation_window, historic_demand_maize  , rate_maize, mu_maize, sigma_maize)
    traffic_projections.append(maize.demand)

### Traffic Scenario (based on median NPQ)

In [4]:
# Calculate the net present quantity of each projection
project_WACC = 0.09
WACC_factor = []
for year in range (len(historic_demand_maize)):
    WACC_factor.append(1.0)
for year in range (simulation_window):
    WACC_factor.append(1/((1+project_WACC)**(year)))

traffic_matrix = np.zeros(shape=(len(traffic_projections), 2))
for i in range(len(traffic_projections)):
    traffic_matrix[i,0] = i
    present_value_volume = []
    for j in range(len(traffic_projections[i])):
        present_value_volume.append(traffic_projections[i][j]*WACC_factor[j])
    traffic_matrix[i,1] = int(np.sum(present_value_volume))

df=pd.DataFrame(traffic_matrix, columns=['Iteration','NPQ'])

# Select the median net present quantity
median_iteration = df.loc[df['NPQ']==np.median(df['NPQ'])].index[0]
maize.demand = traffic_projections[median_iteration]

### Interim traffic projections (in the year 2030)

In [6]:
historic_demand = maize.demand[8:13]
historic_years  = maize.years[8:13]
future_scenario = maize.demand[12:15]
future_years    = maize.years[12:15]

# Maize - Probabilistic demand
rate_maize  = 1.02
mu_maize    = 0.01          # avg bonus rate added to base rate (% points)  - input for random method
sigma_maize = 0.025         # standard deviation of bonus rate (% points)   - input for random method

# Create traffic projections
start_year = 2025
simulation_window = 2
interim_projections = []
interim_years = []
for iterations in range(100):
    scenario = forecast.create_scenario(2025, 3, historic_demand)
    t, demand = scenario.random(rate_maize, mu_maize, sigma_maize)
    interim_years.append(t)
    interim_years[-1]=interim_years[-1][-3:]
    interim_projections.append(demand)
    interim_projections[-1]=interim_projections[-1][-3:]

### Visualizations
- Traffic scenario until now, with trigger placed on future foresight (perfect foresight method)
- Traffic scenario until now, with trigger placed on current demand (current performance trigger) 
- Traffic scenario until now, with trigger placed on forecast based on interim projections (forecast trigger)

### Perfect foresight method

In [126]:
# Visualize traffic scenario up until now
data = []
data.append(
    go.Scatter(
        x = historic_years,
        y = historic_demand,
        name = 'Materialized traffic',
        marker = dict(
            opacity = 0),
        line = dict(
            color = ('rgb(214, 39, 40)'),
            width = 2)))

# Visualize expected traffic scenario based on perfect foresight
data.append(
    go.Scatter(
        x = future_years,
        y = future_scenario,
        name = 'Perfect foresight',
        marker = dict(
            opacity = 0),
        line = dict(
            color = ('rgb(214, 39, 40)'),
            width = 2,
            dash  = 'dot')))
    
# Edit the layout
layout = dict(
            title = 'Future based<br>triggers',
            xaxis = dict(range = [2021,2028]),
            yaxis = dict(title = 'Terminal throughput (t/year)',
                         range = [1310000,1600000]),
            showlegend = False,
            legend=dict(
                x=1.05,
                y=1),
            annotations=[
                dict(
                    x=1,
                    y=-0.12,
                    showarrow=False,
                    text='Source: Own work',
                    xref='paper',
                    yref='paper'),
                dict(
                    x=2027,
                    y=0.3,
                    showarrow=False,
                    text='Traffic volume on which performance<br> trigger is based',
                    yref='paper',
                    xanchor='center',
                    font=dict(
                        size=13))],
            shapes=[
                dict(
                    type='line',
                    x0=2027,
                    y0=0,
                    x1=2027,
                    y1=0.27,
                    yref='paper',
                    line=dict(
                        color='rgb(60, 60, 60)',
                        width=3,
                        dash='dashdot')),
                dict(
                    type='line',
                    x0=2027,
                    y0=0.42,
                    x1=2027,
                    y1=1.05,
                    yref='paper',
                    line=dict(
                        color='rgb(60, 60, 60)',
                        width=3,
                        dash='dashdot'))]

)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Traffic projections')

### Current performance method

In [125]:
# Visualize traffic scenario up until now
data = []
data.append(
    go.Scatter(
        x = historic_years,
        y = historic_demand,
        name = 'Materialized traffic',
        marker = dict(
            opacity = 0),
        line = dict(
            color = ('rgb(214, 39, 40)'),
            width = 2)))
    
# Edit the layout
layout = dict(
            title = 'Current performance<br>triggers',
            xaxis = dict(range = [2021,2028]),
            yaxis = dict(title = 'Terminal throughput (t/year)',
                         range = [1310000,1600000]),
            showlegend = False,
            legend=dict(
                x=1.05,
                y=1),
            annotations=[
                dict(
                    x=1,
                    y=-0.12,
                    showarrow=False,
                    text='Source: Own work',
                    xref='paper',
                    yref='paper'),
                dict(
                    x=2025,
                    y=0.75,
                    showarrow=False,
                    text='Traffic volume on which performance<br> trigger is based',
                    yref='paper',
                    xanchor='center',
                    font=dict(
                        size=13))],
            shapes=[
                dict(
                    type='line',
                    x0=2025,
                    y0=0,
                    x1=2025,
                    y1=0.62,
                    yref='paper',
                    line=dict(
                        color='rgb(60, 60, 60)',
                        width=3,
                        dash='dashdot')),
                dict(
                    type='line',
                    x0=2025,
                    y0=0.77,
                    x1=2025,
                    y1=1.05,
                    yref='paper',
                    line=dict(
                        color='rgb(60, 60, 60)',
                        width=3,
                        dash='dashdot'))]

)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Traffic projections')

### Forecast based method

In [9]:
# Visualize traffic scenario up until now
data = []
data.append(
    go.Scatter(
        x = historic_years,
        y = historic_demand,
        name = 'Materialized traffic',
        marker = dict(
            opacity = 0),
        line = dict(
            color = ('rgb(214, 39, 40)'),
            width = 2)))

# Visualize interim traffic projections
for i in range(len(interim_projections)):
    if i == 0:
        data.append(
            go.Scatter(
                x = interim_years[i],
                y = interim_projections[i],
                name = 'Traffic projections',
                marker = dict(
                    opacity = 0),
                line = dict(
                    shape ='spline',
                    color = ('rgb(215, 215, 215, 0.3)'),
                    width = 2)))
    else:
        data.append(
            go.Scatter(
                x = interim_years[i],
                y = interim_projections[i],
                name = 'Traffic projections',
                marker = dict(
                    opacity = 0),
                showlegend = False,
                line = dict(
                    shape = 'spline',
                    color = ('rgb(215, 215, 215, 0.3)'),
                    width = 2)))
    
# Edit the layout
layout = dict(
            title = 'Forecast based<br>triggers',
            yaxis = dict(title = 'Terminal throughput (t/year)'),
            showlegend = False,
            legend=dict(
                x=1.05,
                y=1),
            annotations=[
                dict(
                    x=1,
                    y=-0.12,
                    showarrow=False,
                    text='Source: Own work',
                    xref='paper',
                    yref='paper'),
                dict(
                    x=2027,
                    y=0.15,
                    showarrow=False,
                    text='Traffic volumes on which performance<br> trigger is based',
                    yref='paper',
                    xanchor='center',
                    font=dict(
                        size=13))],
            shapes=[
                dict(
                    type='line',
                    x0=2027,
                    y0=0,
                    x1=2027,
                    y1=0.12,
                    yref='paper',
                    line=dict(
                        color='rgb(60, 60, 60)',
                        width=3,
                        dash='dashdot')),
                dict(
                    type='line',
                    x0=2027,
                    y0=0.27,
                    x1=2027,
                    y1=1.05,
                    yref='paper',
                    line=dict(
                        color='rgb(60, 60, 60)',
                        width=3,
                        dash='dashdot'))]

)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Traffic projections')